In [ ]:
# @title 1.a Setup environment
!pip install -q --upgrade --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!apt -y install -qq aria2

# Clone the OFFICIAL ComfyUI repository for maximum stability
!git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI
!pip install -q -r /content/ComfyUI/requirements.txt

# GGUF custom nodes from city96
!git clone https://github.com/city96/ComfyUI-GGUF.git /content/ComfyUI/custom_nodes/ComfyUI_GGUF
!pip install -q -r /content/ComfyUI/custom_nodes/ComfyUI_GGUF/requirements.txt

print("------------------------")
print("✅ Environment Setup Complete!")

In [ ]:
# @title 1.b Select Model
selected_model = "base" # -@param["base", "Chroma48"]

models = {
  "Chroma48":{
    "unet":{
      "name": "chroma-unlocked-v48-Q5_K_M.gguf",
      "url": "https://huggingface.co/duyntnet/Chroma-GGUF-All-Versions/resolve/main/chroma-unlocked-v48/",
      "is_gguf": True
    },
    "vae": {
      "name": "ae.safetensors",
      "url": "https://huggingface.co/lodestones/Chroma/resolve/main/",
      "is_gguf": False
      },
    "text_encoder": {
      "name": "t5xxl-encoder-fp32-q6_k.gguf",
      "url": "https://huggingface.co/chatpig/t5-v1_1-xxl-encoder-fp32-gguf/resolve/main/",
      "is_gguf": True
      },
    "type": "chroma"
    },
  "base":{
    "unet":{
      "name": "flux1-schnell-Q4_K_M.gguf",
      "url": "https://huggingface.co/unsloth/FLUX.1-schnell-GGUF/resolve/main/",
      "is_gguf": True
    },
    "vae": {
      "name": "diffusion_pytorch_model.safetensors",
      "url": "https://huggingface.co/tripathiarpan20/FLUX.1-schnell/resolve/main/vae/",
      "is_gguf": False
      },
    "text_encoder": {
      "name": "t5xxl-encoder-fp32-q6_k.gguf",
      "url": "https://huggingface.co/chatpig/t5-v1_1-xxl-encoder-fp32-gguf/resolve/main/",
      "is_gguf": True
      },
    "type": "chroma"
    }
  }

model_entry = models.get(selected_model)
if model_entry:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{model_entry['unet']['url'] + model_entry['unet']['name']}" -d /content/ComfyUI/models/unet -o "{model_entry['unet']['name']}"
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{model_entry['vae']['url'] + model_entry['vae']['name']}" -d /content/ComfyUI/models/vae -o "{model_entry['vae']['name']}"
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{model_entry['text_encoder']['url'] + model_entry['text_encoder']['name']}" -d /content/ComfyUI/models/text_encoders -o "{model_entry['text_encoder']['name']}"
  print("------------------------")
  print("✅ Model Download Complete!")
else:
  print("------------------------")
  print("No valid model selected.")


In [ ]:
# @title 1.c LoRAs and Postprocessing
from google.colab import drive
import os

drive.mount('/content/drive')
!mkdir -p /content/ComfyUI/models/loras

lora_file_root = "/content/drive/MyDrive/AI/LoRAs/Flux.1-Dev"

loras_config = [
  {
    "filename": "sample_lora.safetensors",
    "strength": 1.0
  }
]

for lora_config in loras_config:
  lora_filename = lora_config["filename"]
  !cp "{lora_file_root}/{lora_filename}" "/content/ComfyUI/models/loras/"

# upscaling

upscale_model_name = "RealESRGAN_x4plus.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/Comfy-Org/Real-ESRGAN_repackaged/resolve/main/{upscale_model_name}" -d /content/ComfyUI/models/upscale_models -o {upscale_model_name}

print("✅ LoRAs, Embeddings and Postprocessing Setup Complete!")

In [ ]:
# @title 2. Logic

# Imports
import sys, os, gc, random, pdb
import torch
import logging
from datetime import datetime
from IPython.display import display, Image as IPImage

logging.getLogger().setLevel(logging.ERROR)
sys.path.insert(0, '/content/ComfyUI')

from nodes import (
    UNETLoader, VAELoader, CLIPLoader, LatentUpscale,
    LoraLoaderModelOnly, VAEDecode, KSampler,
    CLIPTextEncode, SaveImage)

from comfy_extras.nodes_upscale_model import (
    UpscaleModelLoader, ImageUpscaleWithModel
)

from comfy_extras.nodes_custom_sampler import (
    CFGGuider, SamplerCustomAdvanced,
    RandomNoise, KSamplerSelect, BasicScheduler
)

from comfy_extras.nodes_cond import (
    T5TokenizerOptions
)

from comfy_extras.nodes_sd3 import (
    EmptySD3LatentImage
)

from custom_nodes.ComfyUI_GGUF.nodes import (
    UnetLoaderGGUF, CLIPLoaderGGUF
)

def generate_images(params):

  model_entry = models.get(selected_model)
  if not model_entry:
    raise ValueError(f"Model not found: The key '{selected_model}' does not exist in the models dictionary.")

  with torch.inference_mode():

    if model_entry["text_encoder"]["is_gguf"]:
      text_encoder = CLIPLoaderGGUF().load_clip(model_entry["text_encoder"]["name"],
                                                              model_entry["type"])[0]
    else:
      text_encoder = CLIPLoader().load_clip(model_entry["text_encoder"]["name"],
                                  type=model_entry["type"], device="default")[0]

    prep_clip = T5TokenizerOptions().set_options(text_encoder, 1, 0)[0]
    #VRAM warning, set_options() clones the text_encoder

    positive_cond = CLIPTextEncode().encode(prep_clip, params['prompt'])[0]
    negative_cond = CLIPTextEncode().encode(prep_clip, params['neg_prompt'])[0]

    print("After loading clip and patching conds")
    print(f"VRAM Used: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

    #need to load model for guider, 2 clips + model = VRAM disaster
    del prep_clip, text_encoder

    if model_entry["unet"]["is_gguf"]:
      model = UnetLoaderGGUF().load_unet(model_entry["unet"]["name"])[0]
    else:
      #need to add weight_dtype to models dictionary
      model = UNETLoader().load_unet(model_entry["unet"]["name"],
                                                 "fp8_e4m3fn")[0]

    for lora_config in loras_config:
      lora_name = lora_config.get("filename")
      strength = float(lora_config.get("strength", 1.0))

      patched_model = LoraLoaderModelOnly().load_lora_model_only(
          model = model,
          lora_name = lora_name,
          strength_model = strength)[0]
      if strength > 0.0:
        #load_lora clones model when strenght > 0.0
        del model
      model = patched_model

    print("After loading unet and loras and patching unet")
    print(f"VRAM Used: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

    guider = CFGGuider().get_guider(model, positive_cond, negative_cond,
                                                       params['cfg'])[0]

    final_width = int(params.get("width", 1024))
    final_height = int(params.get("height", 1024))

    final_steps = int(params.get("steps", 25))

    final_sampler  = params.get("sampler_name", "euler")
    final_scheduler = params.get("scheduler", "beta")

    if params.get("hires_fix", False):
      base_size = 1024

      first_width = (base_size if final_width < final_height
        else final_width * base_size / final_height)
      first_height = (base_size if final_width > final_height
        else final_height * base_size / final_width)

      first_width = round(first_width/8.0) * 8
      first_height = round(first_height/8.0) * 8

      first_steps = round(final_steps * 0.4)
      final_steps = final_steps - first_steps

      first_sampler = "euler"
      first_scheduler = "simple"

    else:
      first_width = final_width
      first_height = final_height
      first_steps = final_steps
      first_sampler = final_sampler
      first_scheduler = final_scheduler

    empty_latent_img = EmptySD3LatentImage().generate(
        first_width, first_height, params['batch_size'])[0]

    noise = RandomNoise().get_noise(params['seed'])[0]
    sampler = KSamplerSelect().get_sampler(first_sampler)[0]
    scheduler = BasicScheduler().get_sigmas(model, first_scheduler, first_steps, denoise=1.0)[0]

    print(f"First Custom Sampler pass with empty latent at W:{first_width}xH:{first_height}")
    print(f"with steps:{first_steps}")

    sampled_latent = SamplerCustomAdvanced().sample(noise, guider, sampler,
                                               scheduler, empty_latent_img)[0]

    print("After first sampling pass:")
    print(f"VRAM Used: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

    if params.get("hires_fix", False):

      upscaled_latent = LatentUpscale().upscale(sampled_latent, "nearest-exact",
                                       final_width, final_height, "disabled")[0]

      print(f"Second KSampler pass with upscaled latent at W:{final_width}xH:{final_height}")
      print(f"with steps:{final_steps}")

      final_latent = KSampler().sample(
        model, params['seed'], final_steps, params['cfg'],
        final_sampler, final_scheduler,
        positive_cond, negative_cond,
        upscaled_latent, denoise=0.5)[0]

      del upscaled_latent

    else:
      final_latent = sampled_latent


    del empty_latent_img
    del model, positive_cond, negative_cond

    if 'patched_model' in locals():
      del patched_model

    if model_entry["vae"]["is_gguf"]:
      raise ValueError(f"There's no VAELoaderGGUF in this library.")
    else:
      vae = VAELoader().load_vae(model_entry["vae"]["name"])[0]

    images = VAEDecode().decode(vae, final_latent);

    print("After loading vae and decoding images")
    print(f"VRAM Used: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

    del vae, sampled_latent, final_latent

    if params.get("upscale_with_model", False):
      print(f"Upscaling with model: {upscale_model_name}")
      upscale_model = UpscaleModelLoader().load_model(upscale_model_name)[0]
      upscaled_images = ImageUpscaleWithModel().upscale(upscale_model, images[0])
      del images, upscale_model
      images = upscaled_images

    return images

def save_and_display_images(tensor_images, generation_params):

  if tensor_images is None or len(tensor_images) == 0:
    print("No image to save or display.")
    return

  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S.%f")[:-3]
  prefix = f"{timestamp}-"

  saved_images = []

  for t_image in tensor_images:
    cpu_image = t_image.detach().cpu()
    s_image = SaveImage().save_images(cpu_image, prefix, prompt=None,
                        extra_pnginfo=generation_params)
    saved_images.append(s_image['ui']['images'][0])

  print(f"Images saved")

  # Display the image in Colab
  for saved_image in saved_images:
    print("\n--- Generated Image ---")
    display(IPImage(filename= os.path.join("/content/ComfyUI/output",
                                           saved_image['subfolder'],
                                           saved_image['filename'])))

    print("\n--- Generation Details ---")
    print(f"Model: {models[selected_model]['unet']['name']}")
    for key, value in generation_params.items():
        print(f"{key.replace('_', ' ').capitalize()}: {value}")
    print("-------------------------")

def clean_memory():
  gc.collect()
  if torch.cuda.is_available():
      torch.cuda.empty_cache()
      torch.cuda.ipc_collect()
  for obj in list(globals().values()):
      if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
          del obj
  gc.collect()

print("✅ Logic Setup Complete!")

In [ ]:
# @title Pre-3. Saved Prompts
sp_select = 1

s_p = ""
s_n_p = ""

if sp_select == 1:
  s_p = ("DSLR photography of a black kitten playing with a ball of yarn")
  s_n_p = ("worst quality, low quality, painting, drawing, sketch, cartoon,"
           "anime, manga, render, CG, 3d, blurry, deformed, disfigured,"
           "mutated, bad anatomy, bad art, watermark, signature, label")    

print("✅ Stored prompt selected!")

In [ ]:
# @title 3. Execute

prompt = "DSLR photography of a black kitten playing with a ball of yarn" # @param {"type":"string"}
if s_p:
  prompt = s_p

neg_prompt = "3D, cartoon, painting, bad quality, low quality" # @param {"type":"string"}
if s_n_p:
  neg_prompt = s_n_p

width = 768 # @param {"type":"slider", "min":"512", "max":"2048", "step": "16"}
height = 1024 # @param {"type":"slider", "min":"512", "max":"2048", "step": "16"}
steps = 6 # @param {"type":"number"}
cfg = 1.5 # @param {"type":"number"}
sampler_name = "euler" # @param ["euler", "euler_cfg_pp", "euler_ancestral", "euler_ancestral_cfg_pp", "heun", "heunpp2","dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_2s_ancestral_cfg_pp", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_cfg_pp", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "dpmpp_3m_sde", "dpmpp_3m_sde_gpu", "ddpm", "lcm", "ipndm", "ipndm_v", "deis", "res_multistep", "res_multistep_cfg_pp", "res_multistep_ancestral", "res_multistep_ancestral_cfg_pp", "gradient_estimation", "gradient_estimation_cfg_pp", "er_sde", "seeds_2", "seeds_3", "sa_solver", "sa_solver_pece"]
scheduler = "beta" # @param ["simple", "sgm_uniform", "karras", "exponential", "ddim_uniform", "beta", "normal", "linear_quadratic", "kl_optimal"]
hires_fix = False # @param{"type":"boolean"}
upscale_with_model = False # @param{"type":"boolean"}
batch_size = 1 # @param {"type":"number"}
seed = 0 # @param {"type":"number"}
clip_skip = 1 # @param {"type" : "number" }

if seed == 0:
  seed = random.randint(0, 2**32 - 1)
print(f"Seed : {seed}")

generation_params = {
    "prompt": prompt,
    "neg_prompt": neg_prompt,
    "width": width,
    "height": height,
    "steps": steps,
    "cfg": cfg,
    "sampler_name": sampler_name,
    "scheduler": scheduler,
    "hires_fix" : hires_fix,
    "upscale_with_model" : upscale_with_model,
    "batch_size": batch_size,
    "seed": seed,
    "clip_skip": clip_skip
}

images = generate_images(generation_params)
save_and_display_images(images, generation_params)

del images
clean_memory()

In [ ]:
# @title Clean Memory
clean_memory()